In [4]:
import utils

In [7]:
"""
This script checks if there are new files in the uploaded folder and processes them.
It computes 3 metrics based on band powers for each signal and uses a new time stamp,
taken as the first for each metric calculation window.
It proceeds by saving the new csv files in the processed folder.
"""

import os
import numpy as np
import csv
from datetime import datetime
import shutil
from scipy.signal import butter, filtfilt

# Importing the function to compute band power
from utils import compute_band_powers  

eeg_files_dir = "Recordings/uploaded" # Replace with the path to your uploaded folder
processed_files_dir = "Recordings/processed" # Replace with the path to your processed folder

# Define experimental parameters
BUFFER_LENGTH = 5  # seconds
EPOCH_LENGTH = 1  # seconds
OVERLAP_LENGTH = 0.8  # seconds
fs = 256  # Hz, sampling frequency for Muse2

# Define bandstop filter parameters: eliminating potential power line noise artifacts from the data
NOTCH_B, NOTCH_A = butter(4, np.array([55, 65]) / (256 / 2), btype='bandstop')

def apply_bandstop_filter(eeg_data):
    """Apply a bandstop filter to EEG data."""
    filtered_data = filtfilt(NOTCH_B, NOTCH_A, eeg_data, axis=0)
    return filtered_data

def compute_epoch_metrics(epoch_timestamps, epochs, fs):
    """Compute 3 metrics based on band powers for each signal
    and return them along with their start timestamps."""
    all_results = []
    
    # The compute_band_powers function returns band powers as [Delta, Theta, Alpha, Beta] for each channel
    for epoch_timestamp, epoch in zip(epoch_timestamps, epochs):
        band_powers = compute_band_powers(epoch, fs)  # Calculate band powers for the epoch
        
        # Calculate metrics
        alpha_relaxation = band_powers[:, 2] / band_powers[:, 0]  # Alpha / Delta
        beta_concentration = band_powers[:, 3] / band_powers[:, 1]  # Beta / Theta
        theta_relaxation = band_powers[:, 1] / band_powers[:, 2]  # Theta / Alpha
        
        # Combine metrics with the timestamp
        metrics = np.hstack((epoch_timestamp, alpha_relaxation, beta_concentration, theta_relaxation))
        all_results.append(metrics)
    
    return np.array(all_results)

def epoch_data(timestamps, eeg_data):
    """Segment EEG data into epochs and return them along with their start timestamps."""
    shift_samples = int((EPOCH_LENGTH - OVERLAP_LENGTH) * fs)
    epoch_samples = int(EPOCH_LENGTH * fs)
    
    epochs = [eeg_data[i:i+epoch_samples] for i in range(
        0, len(eeg_data) - epoch_samples + 1, shift_samples
        )]
    epoch_timestamps = [timestamps[i] for i in range(
        0, len(timestamps) - epoch_samples + 1, shift_samples
        )]
    
    return np.array(epoch_timestamps), np.array(epochs)

def process_eeg_file(file_path):
    """Process an EEG file and save the processed data to a new CSV file."""
    try:
        data = np.genfromtxt(file_path, delimiter=',', skip_header=1)
        timestamps = data[:, 0]
        eeg_data = data[:, 1:-1]  # Exclude the first timestamp column and the last AUX column

        # Apply the bandstop filter to the EEG data
        filtered_eeg_data = apply_bandstop_filter(eeg_data)
        
        epoch_timestamps, epochs = epoch_data(timestamps, filtered_eeg_data)
        processed_data = compute_epoch_metrics(epoch_timestamps, epochs, fs)
        
        # Use the original file's timestamp for the processed file's name
        original_file_name = os.path.basename(file_path)
        output_file = os.path.join(processed_files_dir, f"processed_{original_file_name}")
        save_processed_data(processed_data, output_file)
        
        print(f"Processed and saved: {output_file}")
    except Exception as e:
        print(f"Error processing {file_path}: {e}")


def save_processed_data(processed_data, output_file_path):
    """Save the processed data to a new CSV file."""
    headers = ['Timestamp'] + [f'Ch{ch} Metric {metric}' for ch in range(1, processed_data.shape[1] - 1) for metric in ['Alpha Relaxation', 'Beta Concentration', 'Theta Relaxation']]
    
    with open(output_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in processed_data:
            writer.writerow(row)

def main():
    # Ensure the processed directory exists
    if not os.path.exists(processed_files_dir):
        os.makedirs(processed_files_dir)
        
    unprocessed_files = [os.path.join(eeg_files_dir, f) for f in os.listdir(eeg_files_dir) if f.endswith('.csv')]
    
    for file_path in unprocessed_files:
        print(f"Processing {file_path}...")
        process_eeg_file(file_path)
        # Optionally, move or copy the original file to another directory for archiving
        # shutil.copy(file_path, "<archive_directory>")  # Copies the file to archive directory

if __name__ == "__main__":
    main()


Processing Recordings/uploaded/eeg_data_2024_03_16_12_01_02.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_12_01_02.csv
Processing Recordings/uploaded/eeg_data_2024_03_16_14_42_48.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_14_42_48.csv
Processing Recordings/uploaded/eeg_data_2024_03_16_14_06_24.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_14_06_24.csv
Processing Recordings/uploaded/eeg_data_2024_03_18_15_17_53.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_18_15_17_53.csv
Processing Recordings/uploaded/eeg_data_2024_03_17_14_11_14.csv...
Error processing Recordings/uploaded/eeg_data_2024_03_17_14_11_14.csv: too many indices for array: array is 1-dimensional, but 2 were indexed
Processing Recordings/uploaded/eeg_data_2024_03_19_10_46_07.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_19_10_46_07.csv
Processing Recordings/uploaded/

/var/folders/n1/65g1s9bx2n53jpx4n8scsbzw0000gn/T/ipykernel_36885/1664588688.py:72: UserWarning: genfromtxt: Empty input file: "Recordings/uploaded/eeg_data_2024_03_16_12_17_05.csv"
  data = np.genfromtxt(file_path, delimiter=',', skip_header=1)


Processed and saved: Recordings/processed/processed_eeg_data_2024_03_17_12_10_35.csv
Processing Recordings/uploaded/eeg_data_2024_03_18_15_28_15.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_18_15_28_15.csv
Processing Recordings/uploaded/eeg_data_2024_03_16_10_56_34.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_10_56_34.csv
Processing Recordings/uploaded/eeg_data_2024_03_18_15_17_20.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_18_15_17_20.csv
Processing Recordings/uploaded/eeg_data_2024_03_16_08_39_56.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_08_39_56.csv
Processing Recordings/uploaded/eeg_data_2024_03_18_14_21_40.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_18_14_21_40.csv
Processing Recordings/uploaded/eeg_data_2024_03_16_17_06_53.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_17_06_53.csv
Pro

/var/folders/n1/65g1s9bx2n53jpx4n8scsbzw0000gn/T/ipykernel_36885/1664588688.py:72: UserWarning: genfromtxt: Empty input file: "Recordings/uploaded/eeg_data_2024_03_16_14_40_37.csv"
  data = np.genfromtxt(file_path, delimiter=',', skip_header=1)


Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_08_29_14.csv
Processing Recordings/uploaded/eeg_data_2024_03_17_15_13_55.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_17_15_13_55.csv
Processing Recordings/uploaded/eeg_data_2024_03_16_12_41_29.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_12_41_29.csv
Processing Recordings/uploaded/eeg_data_2024_03_16_09_52_29.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_09_52_29.csv
Processing Recordings/uploaded/eeg_data_2024_03_18_15_03_16.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_18_15_03_16.csv
Processing Recordings/uploaded/eeg_data_2024_03_16_07_49_28.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_07_49_28.csv
Processing Recordings/uploaded/eeg_data_2024_03_16_08_59_44.csv...
Processed and saved: Recordings/processed/processed_eeg_data_2024_03_16_08_59_44.csv
